In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.bandit.algorithms.standard import EpsilonGreedy
from src.bandit.arms.bernoulli import BernoulliArm
from src.bandit.testing.monte_carlo import test_algorithm

In [9]:
x = [0,1,1,1]
m = max(x)
x.index(m)

1

In [ ]:
reward_amounts = [30, 36, 42, 48, 60] # reward from each arm
means = [0.35, 0.33, 0.30, 0.25, 0.20]  # probability of conversion
n_arms = len(reward_amounts)

In [ ]:
test_results = pd.DataFrame(columns=["group","sim_num","time","chosen_arm","reward","cumulative_reward"])

arms = [BernoulliArm(i) for i in means]
algo = EpsilonGreedy(0.4,[],[])
algo.initialize(n_arms)

results = test_algorithm(
    algo=algo, 
    arms=arms, 
    reward_amounts=reward_amounts,
    n_iter=50, 
    horizon=1000
)

data = {
    "sim_num": results[0],
    "time": results[1],
    "chosen_arm": results[2],
    "reward": results[3],
    "cumulative_reward": results[4]
}

result = pd.DataFrame(data = data)

test_results = pd.concat([test_results, result])

In [ ]:
test_results.head(25)

In [ ]:
sns.lineplot(
    x=test_results["time"],
    y=test_results["cumulative_reward"],
    hue=test_results["group"]
)

In [ ]:
test_results.iloc[400:,:].groupby(["group","chosen_arm"]).mean(["reward"])

In [ ]:
sns.histplot(
    data=test_results.query("group=='low engagement'"),
    x="reward",
    hue=test_results.query("group=='low engagement' & reward > 0")["chosen_arm"].astype(str),
    common_bins=True,
    common_norm=False,
    kde=True,
    stat="probability",
    alpha=0.2 
)

In [ ]:
burn_in_period = 200
test_results.query(f"epsilon==0.2 & time>{burn_in_period}")
sns.histplot(
    data=np.log1p(test_results.query(f"epsilon==0.2 & time>{burn_in_period}")),
    x="reward",
    hue="chosen_arm",
    stat="probability",
    common_norm="false",
    common_bins="false"
)

In [ ]:
test_results.query("epsilon==0.2 & time==200").groupby("chosen_arm").mean("reward")

In [ ]:
test_results.query("epsilon==0.4 & time==200")

In [ ]:
sns.lineplot(
    data=test_results, 
    x="time", 
    y="reward",
    hue="epsilon")

##### Generate fake users

##### Initialize algorithm with the best known counts and values

In [1]:
arm_names=["10 then 40", "20 then 40", "10 only", "40 only"]

obs_conv_rates=[0.10, 0, 0.03, 0]
obs_rewards = [75, 70, 90, 60]
starting_counts = [1, 1, 1, 1]
true_conv_rates=[0.15, 0.20, 0.14, 0.17]
true_rewards = [85, 75, 90, 60]

algo = EpsilonGreedy(
    epsilon=0.2,
    n_arms=4,
    rewards= obs_rewards,
    conv_rates=obs_conv_rates,
    counts=starting_counts
)

arms = [BernoulliArm(i) for i in true_conv_rates]

NameError: name 'EpsilonGreedy' is not defined

##### Assign discount for each user

In [81]:
latest_performance = users.\
    groupby(["arm"]).\
        agg(
            {
                "arm": ["count"],
                "conversion": ["mean"],
                "reward": ["mean"]
            }).reset_index()[["arm","conversion","reward"]]

In [84]:
latest_performance

arm       conversion     reward
      count       mean       mean
0   0    86   0.139535  11.860465
1   1     4   0.250000  18.750000
2   2     6   0.000000   0.000000
3   3     4   0.000000   0.000000

In [76]:
latest_performance = users.groupby(["arm"]).mean(["conversion","reward"])
prev_counts = starting_counts
next_counts = [i for i,j in zip(prev_counts, )]

,arm,conversion,reward
0,0,0.139535,11.860465
1,1,0.250000,18.750000
2,2,0.000000,0.000000
3,3,0.000000,0.000000


In [64]:
results = pd.DataFrame(columns=["sim","arm","conversion","reward"])

for i in range(100):
    # generate 100 users
    users = pd.DataFrame(
        data={
            "ID": range(0,100)
        }
    )

    # simulate user outcomes
    users["arm"] = [algo.select_arm() for i in users["ID"]]
    users["conversion"] = [arms[i].draw() for i in users["arm"]]
    users["reward"] = [true_rewards[i]*j for i,j in zip(users["arm"], users["conversion"])]
    latest_performance = users.groupby(["arm"]).mean(["conversion","reward"]).reset_index()[["arm","conversion","reward"]]
    latest_performance["sim"] = i

    # save latest performance
    results = pd.concat([results, latest_performance])
    
    # update assumptions for bandit
    algo = EpsilonGreedy(
        epsilon=0.2,
        n_arms=4,
        rewards= obs_rewards,
        conv_rates=obs_conv_rates,
        counts=[1, 1, 1, 1]
    )

,arm,conversion,reward
0,0,0.139535,11.860465
1,1,0.250000,18.750000
2,2,0.000000,0.000000
3,3,0.000000,0.000000


##### Gather conversion rates and update assumptions

new